In [1]:
from pyspark.sql.session import SparkSession
from pyspark.context import SparkContext
sc = SparkContext('local')

from pyspark.sql import HiveContext
hc = HiveContext(sc)

# Create a Spark session
spark = SparkSession(sc)

# Read Parquet files
#results_csv = "hdfs://172.17.0.2:9000/ingest/results.csv"
#drivers_csv = "hdfs://172.17.0.2:9000/ingest/drivers.csv"
#constructors_csv = "hdfs://172.17.0.2:9000/ingest/constructors.csv"

results_csv = "../data/results.csv"
drivers_csv = "../data/drivers.csv"
constructors_csv = "../data/constructors.csv"

#Load data
df_res = spark.read.option("header", True).csv(results_csv)

df_driv = spark.read.option("header", True).csv(drivers_csv)

df_cons = spark.read.option("header", True).csv(constructors_csv)


23/09/03 20:45:26 WARN Utils: Your hostname, rodrigo-Latitude-5400 resolves to a loopback address: 127.0.1.1; using 192.168.0.177 instead (on interface wlo1)
23/09/03 20:45:26 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/09/03 20:45:27 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
/home/rodrigo/miniconda3/envs/edvai/lib/python3.10/site-packages/pyspark/sql/context.py:720: FutureWarning: HiveContext is deprecated in Spark 2.0.0. Please use SparkSession.builder.enableHiveSupport().getOrCreate() instead.
  warnings.warn(
/home/rodrigo/miniconda3/envs/edvai/lib/python3.10/site-packages/pyspark/sql/context.py:112: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [18]:
df_res = df_res.withColumn("points", df_res["points"].cast("integer"))
df_res.show(5)

+--------+------+--------+-------------+------+----+--------+------------+-------------+------+----+-----------+------------+----------+----+--------------+---------------+--------+
|resultId|raceId|driverId|constructorId|number|grid|position|positionText|positionOrder|points|laps|       time|milliseconds|fastestLap|rank|fastestLapTime|fastestLapSpeed|statusId|
+--------+------+--------+-------------+------+----+--------+------------+-------------+------+----+-----------+------------+----------+----+--------------+---------------+--------+
|       1|    18|       1|            1|    22|   1|       1|           1|            1|    10|  58|1:34:50.616|     5690616|        39|   2|      1:27.452|        218.300|       1|
|       2|    18|       2|            2|     3|   5|       2|           2|            2|     8|  58|     +5.478|     5696094|        41|   3|      1:27.739|        217.586|       1|
|       3|    18|       3|            3|     7|   7|       3|           3|            3|  

In [45]:
#join drivers and results
drivers_res = df_res.join(df_driv, df_res.driverId == df_driv.driverId)
#select columns
drivers_res= drivers_res.select(drivers_res.forename, drivers_res.surname, drivers_res.nationality, drivers_res.points, df_res.driverId)

In [47]:
#group by drivers sum point
most_point = df_res.groupBy("driverId").sum("points").orderBy("sum(points)", ascending=False)
#filter columns > than 0
most_point = most_point.filter(most_point["sum(points)"] > 0)

In [54]:
#Filter drivers with most points
drivers_load = drivers_res.filter(drivers_res.driverId == most_point.driverId).select(drivers_res.forename,drivers_res.surname,drivers_res.nationality,drivers_res.points)

##Creamos una vista con la data filtrada###
drivers_load.createOrReplaceTempView("drivers_load_view")


In [55]:
drivers_load

DataFrame[forename: string, surname: string, nationality: string, points: int]

In [53]:

#Insert data into table
##insertamos el DF filtrado en la tabla tripdata_table
hc.sql("insert into f1.driver_results select * from drivers_load_view;")

+--------+----------+-----------+------+
|forename|   surname|nationality|points|
+--------+----------+-----------+------+
|   Lewis|  Hamilton|    British|    10|
|    Nick|  Heidfeld|     German|     8|
|    Nico|   Rosberg|     German|     6|
|Fernando|    Alonso|    Spanish|     5|
|  Heikki|Kovalainen|    Finnish|     4|
+--------+----------+-----------+------+
only showing top 5 rows



# punto 2 ex 5

In [65]:
total_cons = df_res.filter(df_res.raceId == 318).groupby("constructorId").sum("points").orderBy("sum(points)", ascending=False)
total_cons = total_cons.filter(total_cons["sum(points)"] > 0)

In [70]:
total_cons.show()

+-------------+-----------+
|constructorId|sum(points)|
+-------------+-----------+
|            3|         14|
|            6|          9|
|            1|          2|
|           22|          1|
+-------------+-----------+



In [79]:
df_cons.show(2)

+-------------+--------------+----------+-----------+--------------------+
|constructorId|constructorRef|      name|nationality|                 url|
+-------------+--------------+----------+-----------+--------------------+
|            1|       mclaren|   McLaren|    British|http://en.wikiped...|
|            2|    bmw_sauber|BMW Sauber|     German|http://en.wikiped...|
+-------------+--------------+----------+-----------+--------------------+
only showing top 2 rows



In [80]:
#type of values in df_cons
df_cons.dtypes

[('constructorId', 'string'),
 ('constructorRef', 'string'),
 ('name', 'string'),
 ('nationality', 'string'),
 ('url', 'string')]

In [96]:
#left join between df_cons and total_cons
df_cons_filt = total_cons.join(df_cons, df_cons.constructorId == total_cons.constructorId, how="left").select(df_cons.constructorRef, df_cons.name, df_cons.nationality, df_cons.url, total_cons["sum(points)"].alias("total_points"))


In [97]:
df_cons_filt.show()

+--------------+--------+-----------+--------------------+------------+
|constructorRef|    name|nationality|                 url|total_points|
+--------------+--------+-----------+--------------------+------------+
|      williams|Williams|    British|http://en.wikiped...|          14|
|      benetton|Benetton|    Italian|http://en.wikiped...|           1|
|       ferrari| Ferrari|    Italian|http://en.wikiped...|           9|
|       mclaren| McLaren|    British|http://en.wikiped...|           2|
+--------------+--------+-----------+--------------------+------------+



In [ ]:
##Creamos una vista con la data filtrada###
drivers_load.createOrReplaceTempView("drivers_load_view")

#Insert data into table
##insertamos el DF filtrado en la tabla tripdata_table
hc.sql("insert into f1.driver_results select * from drivers_load_view;")